In [95]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [96]:
# read in all words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [97]:
len(words)

32033

In [98]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [99]:
# built the dataset
# train split , dev split, test split
# 0.8, 0.1, 0.1
def build_dataset(words):
    block_size = 3 # context length: number of words considered in context
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append
            
    X = torch.tensor(X) 
    Y = torch.tensor(Y) 
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:]) 

In [100]:
# --------------------------------------- respectable :) ---------------------
g = torch.Generator().manual_seed(2147483647)
C = torch.randn([27, 2])    # each character means a point in vector space
W1 = torch.randn([6, 100])  # 100 neurons
b1 = torch.randn(100)
W2 = torch.randn([100, 27])
b2 = torch.randn(27)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

In [101]:
sum(p.nelement() for p in parameters) # number of parameters in total

3481

In [110]:
for i in range(10000):
    # minibatch
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # forward pass
    emb = C[Xtr[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    
    logits = h @ W2 + b2
    #counts = logits.exp()
    #probs = counts / counts.sum(1, keepdim=True)
    #loss = -probs[torch.arange(X.shape[0]), Y].log().mean()
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    lr = 0.01
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
    # lri.append(lr)
    # lossi.append(loss.item())

print(loss.item())

2.3746120929718018


In [111]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
print(loss.item())

2.3392653465270996


In [112]:
emb = C[Xte] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yte)
print(loss.item())

2.338972330093384


In [123]:
g = torch.Generator().manual_seed(2147483647 + 10)

# sample from the model
for _ in range(20):
    out = []
    context = [0] * 3
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print(''.join(itos[i] for i in out))

mria.
amyan.
seel.
ndhayah.
rethan.
endrlen.
aderedielii.
shi.
jenleie.
seso.
aar.
ely.
korlara.
noshugtrishianest.
jair.
jelilanteulanu.
zeygen.
jyshue.
eha.
kay.
